# Running the Tweeted Configurations

This notebook runs the latest tweeted battery model configuration, and can also be modified and played with. The notebook requires no user input (unless you want to modify some stuff).

Access this on Google Colab [here](https://colab.research.google.com/github/pybamm-team/BattBot/blob/main/bot/run-simulation.ipynb).

In [1]:
import logging
import os

import matplotlib.pyplot as plt
import numpy as np
import pybamm
import requests

%pip install git+https://github.com/pybamm-team/PyBaMM@develop -q  # install the develop branch of PyBaMM


# logging configs
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

We start by opening config.txt, where the most recent tweeted configuration is stored. Next we read that data and convert it into a python dictionary.

In [2]:
# open the file and read the python dictionary for configurations
url = "https://raw.githubusercontent.com/pybamm-team/BattBot/main/bot/config.txt"
r = requests.get(url, allow_redirects=True)
open("config.txt", "wb").write(r.content)
f = open("config.txt")
config = eval(f.read())

# varied parameter values, you can play with them too
param_values = config["varied_values"]

# extracting the models from "config" dictionary
models = []
if "DFN" in config["model"]:
    models.append(pybamm.lithium_ion.DFN(options=config["model options"]))
if "spm.SPM" in config["model"]:
    models.append(pybamm.lithium_ion.SPM(options=config["model options"]))
if "spme.SPMe" in config["model"]:
    models.append(pybamm.lithium_ion.SPMe(options=config["model options"]))

# closing the file
f.close()
config
# here you can update the config dictionary by doing something like this -
# config.update({
#   "key": value
# })
# the default dictionary which changes every time a plot is tweeted is printed below

{'model': '{0: <pybamm.models.full_battery_models.lithium_ion.spm.SPM object at 0x7f170803bd90>}',
 'model options': None,
 'chemistry': {'chemistry': 'lithium_ion',
  'cell': 'LGM50_Chen2020',
  'negative electrode': 'graphite_Chen2020',
  'separator': 'separator_Chen2020',
  'positive electrode': 'nmc_Chen2020',
  'electrolyte': 'lipf6_Nyman2008',
  'experiment': '1C_discharge_from_full_Chen2020',
  'sei': 'example',
  'citation': 'Chen2020'},
 'is_experiment': False,
 'cycle': None,
 'number': None,
 'is_comparison': True,
 'param_to_vary': 'Electrode height [m]',
 'varied_values': [0.10317, 0.12472, 0.11773]}

In [3]:
# a class that scales functional parameters


class FunctionLike:
    "Behaves like a function but saves fun and parameter"

    def __init__(self, fun, parameter):
        self.fun = fun
        self.parameter = parameter

    def __call__(self, *args):
        return self.parameter * self.fun(*args)

    def __str__(self):
        return str(self.parameter)

The following cell is meant to plot summary variables, if they weren't tweeted in the latest tweet then you can skip this. (Note: running the cell won't make a difference if the last tweet was not about summary variables)

In [4]:
pybamm.set_logging_level("NOTICE")

# if summary variables were tweeted
if config["is_experiment"] and not config["is_comparison"]:
    # don't terminate early if Ai2020 parameter sets were used
    if config["chemistry"] == "Ai2020":
        experiment = pybamm.Experiment(config["cycle"] * config["number"])
    else:
        experiment = pybamm.Experiment(
            config["cycle"] * config["number"], termination="80% capacity"
        )

    parameter_values = pybamm.ParameterValues(config["chemistry"])

    # update parameter sets for Mohtat2020 chemistry
    if config["chemistry"] == "Mohtat2020":

        def lico2_volume_change_Ai2020(sto):
            omega = pybamm.Parameter(
                "Positive electrode partial molar volume [m3.mol-1]"
            )
            c_p_max = pybamm.Parameter(
                "Maximum concentration in positive electrode [mol.m-3]"
            )
            t_change = omega * c_p_max * sto
            return t_change

        def graphite_volume_change_Ai2020(sto):
            p1 = 145.907
            p2 = -681.229
            p3 = 1334.442
            p4 = -1415.710
            p5 = 873.906
            p6 = -312.528
            p7 = 60.641
            p8 = -5.706
            p9 = 0.386
            p10 = -4.966e-05
            t_change = (
                p1 * sto**9
                + p2 * sto**8
                + p3 * sto**7
                + p4 * sto**6
                + p5 * sto**5
                + p6 * sto**4
                + p7 * sto**3
                + p8 * sto**2
                + p9 * sto
                + p10
            )
            return t_change

        parameter_values.update(
            {
                # mechanical properties
                "Positive electrode Poisson's ratio": 0.3,
                "Positive electrode Young's modulus [Pa]": 375e9,
                "Positive electrode reference concentration for free of deformation [mol.m-3]": 0,  # noqa
                "Positive electrode partial molar volume [m3.mol-1]": -7.28e-7,
                "Positive electrode volume change": lico2_volume_change_Ai2020,
                "Negative electrode volume change": graphite_volume_change_Ai2020,
                # Loss of active materials (LAM) model
                "Positive electrode LAM constant exponential term": 2,
                "Positive electrode critical stress [Pa]": 375e6,
                # mechanical properties
                "Negative electrode Poisson's ratio": 0.3,
                "Negative electrode Young's modulus [Pa]": 15e9,
                "Negative electrode reference concentration for free of deformation [mol.m-3]": 0,  # noqa
                "Negative electrode partial molar volume [m3.mol-1]": 3.1e-6,
                # Loss of active materials (LAM) model
                "Negative electrode LAM constant exponential term": 2,
                "Negative electrode critical stress [Pa]": 60e6,
                # Other
                "Cell thermal expansion coefficient [m.K-1]": 1.48e-6,
                "SEI kinetic rate constant [m.s-1]": 1e-15,
                "Positive electrode LAM constant propotional term": 1e-3,
                "Negative electrode LAM constant propotional term": 1e-3,
                "EC diffusivity [m2.s-1]": 2e-18,
            },
            check_already_exists=False,
        )

    param_list = []
    labels = []
    solutions = []

    for i in range(0, len(config["varied_values"])):
        # copy the original values and append them in the list
        param_list.append(parameter_values.copy())

        # change a parameter value
        param_list[i][config["param_to_vary"]] = config["varied_values"][i]

        logger.info(config["param_to_vary"] + ": " + str(param_values[i]))

        labels.append(config["param_to_vary"] + ": " + str(param_values[i]))

        sim = pybamm.Simulation(
            model=models[0],
            experiment=experiment,
            parameter_values=param_list[i],
        )

        if config["chemistry"] == "Ai2020":
            sim.solve(calc_esoh=False)
        elif config["chemistry"] == "Mohtat2020":
            sim.solve(initial_soc=1)
        else:
            sim.solve()
        solution = sim.solution
        solutions.append(solution)

    # defining the summary variables
    if config["chemistry"] == "Ai2020":
        vars_to_plot = [
            "Measured capacity [A.h]",
            "Loss of lithium inventory [%]",
            "Loss of active material in negative electrode [%]",
            "Loss of active material in positive electrode [%]",
        ]
    else:
        vars_to_plot = [
            "Capacity [A.h]",
            "Loss of lithium inventory [%]",
            "Loss of active material in negative electrode [%]",
            "Loss of active material in positive electrode [%]",
            "x_100",
            "x_0",
            "y_100",
            "y_0",
        ]

    length = len(vars_to_plot)
    n = int(length // np.sqrt(length))
    m = int(np.ceil(length / n))

    # plotting the summary variables
    fig, axes = plt.subplots(n, m, figsize=(15, 8))
    for var, ax in zip(vars_to_plot, axes.flat):
        for solution in solutions:
            ax.plot(
                solution.summary_variables["Cycle number"],
                solution.summary_variables[var],
            )
        ax.set_xlabel("Cycle number")
        ax.set_ylabel(var)
        ax.set_xlim([1, solution.summary_variables["Cycle number"][-1]])
    fig.tight_layout()
    fig.legend(labels, loc="lower left", bbox_to_anchor=(0.77, -0.08))
    plt.show()

The following cell is meant to create a slider plot for the configurations with no experiment, you can skip this if the latest tweeted GIF had no experiment. (Note: running the cell won't make a difference if the last tweet was not about comparing models with no experiment and with different configurations)

In [5]:
# if it is a comparison plot but has no experiment
if config["is_comparison"] and not config["is_experiment"]:
    # declaring the parameter values
    parameter_values = pybamm.ParameterValues(config["chemistry"])
    param_list = []
    labels = []

    if isinstance(param_values, dict):
        parameter_values["Ambient temperature [K]"] = param_values[
            "Ambient temperature [K]"
        ]
        parameter_values["Current function [A]"] = param_values["Current function [A]"]
    else:
        # if the param_value list is populated then vary the parameter
        for i in range(0, len(param_values)):
            # copy the original values and append them in the list
            param_list.append(parameter_values.copy())

            # change a parameter value
            if callable(param_list[i][config["param_to_vary"]]):
                param_list[i][config["param_to_vary"]] = FunctionLike(
                    param_list[i][config["param_to_vary"]], param_values[i]
                )
            else:
                param_list[i][config["param_to_vary"]] = param_values[i]

            logger.info(config["param_to_vary"] + ": " + str(param_values[i]))

            labels.append(config["param_to_vary"] + ": " + str(param_values[i]))

    # find the minimum value if "Current function [A]" is varied
    if config["param_to_vary"] == "Current function [A]":
        min_param_value = min(config["param_to_vary"])

    if "Current function [A]" in param_values:
        min_param_value = param_values["Current function [A]"]

    models_for_comp = dict(list(enumerate(models)))
    if len(param_list) != 0:
        parameter_values_for_comp = dict(list(enumerate(param_list)))
    else:
        parameter_values_for_comp = dict(list(enumerate([parameter_values])))

    # using BatchStudy for the comparison plots
    s = pybamm.BatchStudy(
        models=models_for_comp,
        parameter_values=parameter_values_for_comp,
        permutations=True,
    )

    # changing the t_eval if "Current function [A]" is being varied
    if (
        config["param_to_vary"] == "Current function [A]"
        or "Current function [A]" in param_values
    ):
        factor = min_param_value / parameter_values["Current function [A]"]
        t_end = (1 / factor * 1.1) * 3600
    else:
        # default t_end
        t_end = 3700

    # solving
    if config["chemistry"] == "Ai2020":
        s.solve([0, t_end], calc_esoh=False)
    else:
        s.solve([0, t_end])

    # plotting
    if len(param_list) != 0:
        s.plot(labels=labels)
    else:
        s.plot()

2021-09-30 23:58:43,216 - [INFO] <ipython-input-5-c8a37ea25e7e>.<module>(33): Electrode height [m]: 0.10317
2021-09-30 23:58:43,221 - [INFO] <ipython-input-5-c8a37ea25e7e>.<module>(33): Electrode height [m]: 0.12472
2021-09-30 23:58:43,221 - [INFO] <ipython-input-5-c8a37ea25e7e>.<module>(33): Electrode height [m]: 0.11773


interactive(children=(FloatSlider(value=0.0, description='t', max=1.0277777777777777, step=0.01027777777777777…

The following cell is meant to create a slider plot for the configurations with experiment, you can skip this if the latest tweeted GIF had no experiment. (Note: running the cell won't make a difference if the last tweet was not about comparing a single experiment with different configurations)

In [6]:
# if it is a comparison plot and has an experiment
if config["is_comparison"] and config["is_experiment"]:
    # declaring the parameter values
    parameter_values = pybamm.ParameterValues(config["chemistry"])
    param_list = []
    labels = []

    # if the param_value list is populated then vary the parameter
    if isinstance(param_values, dict):
        parameter_values["Ambient temperature [K]"] = param_values[
            "Ambient temperature [K]"
        ]
    else:
        for i in range(0, len(param_values)):
            # copy the original values and append them in the list
            param_list.append(parameter_values.copy())

            # change a parameter value
            if callable(param_list[i][config["param_to_vary"]]):
                param_list[i][config["param_to_vary"]] *= param_values[i]
            else:
                param_list[i][config["param_to_vary"]] = param_values[i]

            logger.info(config["param_to_vary"] + ": " + str(param_values[i]))

            labels.append(config["param_to_vary"] + ": " + str(param_values[i]))

    models_for_comp = dict(list(enumerate(models)))
    experiment = dict(
        list(enumerate([pybamm.Experiment(config["cycle"] * config["number"])]))
    )

    # using BatchStudy for the comparison plots
    if len(param_list) != 0:
        parameter_values_for_comp = dict(list(enumerate(param_list)))
    else:
        parameter_values_for_comp = dict(list(enumerate([parameter_values])))
    s = pybamm.BatchStudy(
        models=models_for_comp,
        parameter_values=parameter_values_for_comp,
        experiments=experiment,
        permutations=True,
    )

    # solving
    if config["chemistry"] == "Ai2020":
        s.solve(calc_esoh=False)
    else:
        s.solve()

    # plotting
    if len(param_list) != 0:
        s.plot(labels=labels)
    else:
        s.plot()

In [7]:
pybamm.print_citations()

[1] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[2] Chang-Hui Chen, Ferran Brosa Planella, Kieran O'Regan, Dominika Gastol, W. Dhammika Widanage, and Emma Kendrick. Development of Experimental Techniques for Parameterization of Multi-scale Lithium-ion Battery Models. Journal of The Electrochemical Society, 167(8):080534, 2020. doi:10.1149/1945-7111/ab9050.
[3] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[4] Scott G. Marquis, Valentin Sulzer, Robert Timms, Colin P. Please, and S. Jon Chapman. An asymptotic derivation of a single particle model with elect